In [136]:
from selenium.webdriver.chrome.service import Service
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep

In [137]:
driver = webdriver.Chrome(
)
driver.get("https://judgment.judicial.gov.tw/FJUD/data.aspx?ty=JD&id=TCHM%2c113%2c%e4%b8%8a%e8%a8%b4%2c213%2c20240424%2c1&ot=in")

sleep(1)

In [138]:
## 找找看相關判決欄位，偽元素可能是動態生成的，試試看等待仔入之後抓不抓地到
##>> OK!!!!!所以有偽元素(::after, ::maker)必須使用動態的抓法，因為他是利用javascript生成的用bs4抓不到(bs4拿到的是最初要求的那份html文件，此時這些偽元素欄位尚未被生成)
some = driver.find_element(By.CSS_SELECTOR, 'ul.rela-law')
some.text

'刑事訴訟法 第 159.1、369 條（106.11.16）\n中華民國刑法 第 1、13、25、38、62、95、271 條（105.11.30）'

In [139]:
html  = driver.page_source
html

'<html lang="zh-TW"><head><meta http-equiv="Content-Type" content="text/html; charset=utf-8"><meta http-equiv="X-UA-Compatible" content="IE=edge"><meta http-equiv="CACHE-CONTROL" content="NO-CACHE">\n    <meta property="og:image" content="https://judgment.judicial.gov.twfblogo.png">\n    <title>\n\t臺灣高等法院 臺中分院 113 年度上訴字第 213 號刑事判決\n</title><link rel="stylesheet" href="../css/bootstrap.min.css"><link rel="stylesheet" href="../css/font-awesome.min.css"><link rel="stylesheet" href="../css/iframe.css">\n    <script type="text/javascript" async="" src="https://www.google-analytics.com/analytics.js"></script><script type="text/javascript" async="" src="https://www.googletagmanager.com/gtag/js?id=G-X3FQ17YF4M&amp;l=dataLayer&amp;cx=c"></script><script type="text/javascript" src="/js/jquery-3.6.0.min.js"></script>    \n    <script type="text/javascript" src="/js/isMobile.js"></script>\n    <!--引用司法院內網的網頁字型 javascript -->\n    <script type="text/javascript" src="https://tpjwebfont2.judicial.gov

In [140]:
import requests as req
from bs4 import BeautifulSoup as bs
from pprint import pprint

In [141]:
soup = bs(html, "lxml") 

In [142]:
info_list = []
# 案件標題id, 有四個標籤都是使用此選擇器，包含主文....(第四個)，那就抓這個就好了啊...
all = soup.select("div.col-td")
info_list.append({
    "case_id":all[0].get_text().strip(),
    "date":all[1].get_text().strip(),
    "simple_reason":all[2].get_text().strip(),
    "context":all[3].get_text()
})

In [143]:
info_list

[{'case_id': '臺灣高等法院 臺中分院 113 年度上訴字第 213 號刑事判決',
  'date': '民國 113 年 04 月 24 日',
  'simple_reason': '殺人未遂等',
  'context': '\n\n\n\n\xa0臺灣高等法院臺中分院刑事判決113年度上訴字第213號上\xa0 訴\xa0 人\xa0 臺灣臺中地方檢察署檢察官上\xa0 訴\xa0 人\xa0\xa0即\xa0 被\xa0 告\xa0 HOANG VAN CANH（中文名：黃文景）0000000000000000000000000000000000000000000000000000000000000000指定辯護人\xa0 本院公設辯護人郭博益上列上訴人等因被告殺人未遂等案件，不服臺灣臺中地方法院112年度訴字第1877號，中華民國113年1月17日第一審判決（起訴案號：臺灣臺中地方檢察署112年度偵字第38829號），提起上訴，本院判決如下：\xa0 \xa0 主\xa0 文原判決撤銷。HOANG VAN CANH（中文姓名：黃文景）犯殺人未遂罪，處有期徒刑伍年拾月。並於刑之執行完畢或赦免後，驅遂出境。扣案之折疊刀壹把沒收之。\xa0 \xa0 事\xa0 實一、HOANG VAN CANH（越南籍；中文姓名：黃文景，下稱黃文景\xa0 \xa0 ）與SAMSURI （印尼籍；中文姓名：蘇立；下稱蘇立)均為逃逸外勞，蘇立先找到中部○○水電行老闆願意雇用，黃文景其後才找到同一雇主願意雇用，兩人都是非法打工，該水電行老闆因承攬臺中市梧棲區八德路1 段與八德二路交岔路口「遠雄幸福成- 遠雄之星9 」建築工地（下稱本案工地) 之水電小包工作，老闆先後指派蘇立、黃文景到本案工地做水電，但因為是非法打工，為躲避警察及勞動檢查，只能每天下午三點半集合上工，趁傍晚及夜間工作。民國（下同）112年8月2日（週三）22時30分許，在本案工地內，蘇立與黃文景因工作事項發生摩擦，兩人又語言不通，蘇立先出手抓住黃文景衣領，經旁人勸阻後始鬆手，然又出手推及黃文景，黃文景竟基於殺人之未必故意，突然自隨身之包包內取出折疊刀刺向蘇立左胸一刀（深度5公分、深及心包膜），蘇立遇刺伸出右手抓住刀刃，黃文景將刀刃抽離，蘇立發現自己流血，遂撿拾一旁之水管攻擊黃文景，黃文景伸手隔擋，蘇

In [144]:
lines = soup.select_one('ul.rela-law')
# for i in lines:
#     print(i.get_text())
lines

<ul class="rela-law"><li>刑事訴訟法 第 159.1、369 條（106.11.16）</li><li>中華民國刑法 第 1、13、25、38、62、95、271 條（105.11.30）</li></ul>

In [145]:
# 列一下要爬甚麼資訊，什麼資訊可以在這一步就出分
# 抓全部的內文: 因偽元素的設定沒有章法，不能用同樣css抓到想要的欄位(主文, 律師, 書記官...)
# 法官欄位蠻穩定的就先挑
# soup.select_one("div.col-td")有四個標籤都是使用此選擇器，包含主文....(第四個)，那就抓這個就好了啊...

In [146]:
# 法官(分兩個，因為標籤的緣故)
# 1.審判長
main_judge = soup.select_one("div[style='text-indent: -216px; padding-left: 216px;']")
main_judge = main_judge.get_text()

In [147]:
# 2.其他法官
# 注意屬性間的空格在html中代表的是套用好幾個不同屬性值，但在使用css selector時要用class要用.串起來
other_judges = soup.select("div[style='text-indent: -456px; padding-left: 456px;']")
judges = []
for i in other_judges:
    print(2)
    judges.append(i.get_text())

2
2


In [148]:
# 歷審裁判 history_judgement
history_judgement = soup.select("div#JudHis div.panel-body ul li")
history_judgement[0].get_text()
# history_judgement_url = soup.select("div#JudHis div.panel-body ul li a")
# history_judgement_url[0]['href']

# for ele in history_judgement: 
#     c = ele.select_one('a[href]') #使用選擇器時[屬性]屬性不要加引號 #直接用這個往下找url就好
#     print(c['href']) #取值得時候就要，且必須這樣取才會是網址喔

history_judgement_list = []
for index in range(0, len(history_judgement)):
    history_judgement_list.append({
        "case_id":history_judgement[index].get_text(),
        "url":"https://judgment.judicial.gov.tw/FJUD/"+history_judgement[index].select_one('a')['href']
    })


# history_judgement_list = []
# history_judgement_url_list = []

# for i in history_judgement:
#     history_judgement_list.append(i.get_text())
# for i in history_judgement_url:
#     history_judgement_url_list.append(i['href'])

# combine = []

# for i in range(0, len(history_judgement)):
#     combine.append({
#         "case_id":history_judgement[i],
#         "url":history_judgement_url[i]
#     })
# history_judgement_list = []
# for a in history_judgement:
#     for b in history_judgement_url:
#         history_judgement_list.append({
#             "case_id":a.get_text(),
#             "url":b['href']
#         })

# for ele in history_judgement:
#     history_judgement_dict['case_id'] = ele.get_text()

# for i in len(history_judgement):
#     history_judgement_dict['case_id'] = history_judgement[i].get_text()
#     history_judgement_dict['url'] = history_judgement_url[i]['href']

In [149]:
history_judgement_list

[{'case_id': '臺灣臺中地方法院 112 年度 訴 字第 1877 號裁定(112.12.14)',
  'url': 'https://judgment.judicial.gov.tw/FJUD/data.aspx?ty=JD&id=TCDM%2c112%2c%e8%a8%b4%2c1877%2c20231214%2c1'},
 {'case_id': '臺灣臺中地方法院 112 年度 訴 字第 1877 號判決(113.01.17)',
  'url': 'https://judgment.judicial.gov.tw/FJUD/data.aspx?ty=JD&id=TCDM%2c112%2c%e8%a8%b4%2c1877%2c20240117%2c2'},
 {'case_id': '臺灣高等法院 臺中分院 113 年度 上訴 字第 213 號判決(113.04.24)',
  'url': 'https://judgment.judicial.gov.tw/FJUD/data.aspx?ty=JD&id=TCHM%2c113%2c%e4%b8%8a%e8%a8%b4%2c213%2c20240424%2c1'}]

In [150]:
info_list.append({
    "case_id":all[0].get_text().strip(),
    "date":all[1].get_text().strip(),
    "simple_reason":all[2].get_text().strip(),
    "context":all[3].get_text(),
    "laws":lines,
    "judges":[main_judge, judges],
    "history_judgement":history_judgement_list
})

In [151]:
info_list

[{'case_id': '臺灣高等法院 臺中分院 113 年度上訴字第 213 號刑事判決',
  'date': '民國 113 年 04 月 24 日',
  'simple_reason': '殺人未遂等',
  'context': '\n\n\n\n\xa0臺灣高等法院臺中分院刑事判決113年度上訴字第213號上\xa0 訴\xa0 人\xa0 臺灣臺中地方檢察署檢察官上\xa0 訴\xa0 人\xa0\xa0即\xa0 被\xa0 告\xa0 HOANG VAN CANH（中文名：黃文景）0000000000000000000000000000000000000000000000000000000000000000指定辯護人\xa0 本院公設辯護人郭博益上列上訴人等因被告殺人未遂等案件，不服臺灣臺中地方法院112年度訴字第1877號，中華民國113年1月17日第一審判決（起訴案號：臺灣臺中地方檢察署112年度偵字第38829號），提起上訴，本院判決如下：\xa0 \xa0 主\xa0 文原判決撤銷。HOANG VAN CANH（中文姓名：黃文景）犯殺人未遂罪，處有期徒刑伍年拾月。並於刑之執行完畢或赦免後，驅遂出境。扣案之折疊刀壹把沒收之。\xa0 \xa0 事\xa0 實一、HOANG VAN CANH（越南籍；中文姓名：黃文景，下稱黃文景\xa0 \xa0 ）與SAMSURI （印尼籍；中文姓名：蘇立；下稱蘇立)均為逃逸外勞，蘇立先找到中部○○水電行老闆願意雇用，黃文景其後才找到同一雇主願意雇用，兩人都是非法打工，該水電行老闆因承攬臺中市梧棲區八德路1 段與八德二路交岔路口「遠雄幸福成- 遠雄之星9 」建築工地（下稱本案工地) 之水電小包工作，老闆先後指派蘇立、黃文景到本案工地做水電，但因為是非法打工，為躲避警察及勞動檢查，只能每天下午三點半集合上工，趁傍晚及夜間工作。民國（下同）112年8月2日（週三）22時30分許，在本案工地內，蘇立與黃文景因工作事項發生摩擦，兩人又語言不通，蘇立先出手抓住黃文景衣領，經旁人勸阻後始鬆手，然又出手推及黃文景，黃文景竟基於殺人之未必故意，突然自隨身之包包內取出折疊刀刺向蘇立左胸一刀（深度5公分、深及心包膜），蘇立遇刺伸出右手抓住刀刃，黃文景將刀刃抽離，蘇立發現自己流血，遂撿拾一旁之水管攻擊黃文景，黃文景伸手隔擋，蘇